In [1]:
import sqlite3
import pandas as pd


In [2]:
con = sqlite3.connect("D:/001_Projects/09 - Cargo TIA/03 Aimsun/Model/502942 ACSA Cargo Precinct V12.sqlite")
cur = con.cursor()


In [24]:
AM = 69879
PM = 69889

#AM = 64926
#PM = 64938


In [25]:
turn = pd.read_sql_query('select * from MITURN WHERE did == %d OR did == %d' % (AM,PM), con)
node = pd.read_sql_query('select * from HCMMINODE WHERE did == %d OR did == %d' % (AM,PM), con)
approach = pd.read_sql_query('select * from HCMMIAPPROACH WHERE did == %d OR did == %d' % (AM,PM), con)

In [26]:
turn = turn[(turn.sid == 0)]

turn2 = turn
node2 = node
approach2 = approach

turn2 = turn2[turn2['ent'].between(2, 5)]

turn3 = turn2.groupby(['did','oid']).agg({'dtime':'max',
                                          'count':'sum'}).reset_index()

turn4 = turn3.pivot(index ='oid', columns ='did', values =['dtime','count']).reset_index()

node3 = node2.groupby(['did','oid'])[['los']].max().reset_index()
node4 = node3.pivot(index ='oid', columns ='did', values =['los']).reset_index()

approach3 = approach2.groupby(['did','oid'])[['los','queue_delay', 'queue_length', 'queue_overflow']].max().reset_index()
approach4 = approach3.pivot(index ='oid', columns ='did', values =['los','queue_delay', 'queue_length', 'queue_overflow']).reset_index()

#turn3.head(15)


In [27]:
approach4['queue_delay'].mean()

did
69879    40.645430
69889    98.367428
dtype: float64

In [28]:
node4['los'].mean()

did
69879    2.447368
69889    2.500000
dtype: float64

In [29]:
turn4.to_csv('D:/2029 PWV15 K90S turn.csv')
node4.to_csv('D:/2029 PWV15 K90S node.csv')
approach4.to_csv('D:/2029 PWV15 K90S approach.csv')